<a href="https://colab.research.google.com/github/spoer13/jambu/blob/master/Fill_in_Daily_Analysis_pred_source_from_REST_210613.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import the libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from google.colab import drive
#mount gdrive
drive.mount('/content/gdrive')

import gspread
from oauth2client.client import GoogleCredentials
#Read data from google spread

from google.colab import auth
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())

import requests  # Import the requests library

Mounted at /content/gdrive


In [ ]:
#Load Model for prediction

#model1 = tf.keras.models.load_model('/content/gdrive/MyDrive/saved_model/model_210609c')
model1 = tf.keras.models.load_model('/content/gdrive/MyDrive/saved_model/model_210610e')
model1.summary()


Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 500)               1006000   
_________________________________________________________________
dense_10 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 101       
Total params: 1,056,201
Trainable params: 1,056,201
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Get the input data from google spreadsheet 'Input Spreadsheet for Pre'
#wb = gc.open_by_key('1WsoN1hkwTGPShVaY2lJVaPb8XKgnp3FwxrxpZclFWKk')

#Get the input data from google spreadsheet 'mmyy - Analysis'
wb = gc.open_by_key('1XNGejQ8-64jyUReMwDvPcjxZfZzb2_fNPsjww2tX1W4')
ws = wb.get_worksheet(0)     
tickName = []
rows = ws.get_all_values()[1:]
rowSize = len(rows)
for tick in range(13,rowSize):                   # range = start of tick row - rowSize [zero idx]
  if(rows[tick][0] != ""):
    tickName.append((rows[tick][0][-4:] + ".JK"))

snum = len(tickName)
# snum = 3
# count = 0
for count in range(0,snum):
  # ws = wb.get_worksheet(sheetNumber)
  # ws = wb.get_worksheet(count)              #parameter = sheet number starting from zero
  # tickName = ws.title
  print(tickName[count])

  # Query URL
  url = ('https://query1.finance.yahoo.com/v8/finance/chart/'+ tickName[count] +
        '?region=US&lang=en-US&includePrePost=false&interval=30m&useYfid=true&range=5d&corsDomain=finance.yahoo.com')
  # print(url)

  response = requests.get(url)  # Make a GET request to the URL
  payload = response.json()  # Parse `response.text` into JSON

  #take data up to yesterday  (strip 14 last records) - when you run the pred next day
  # close = payload["chart"]["result"][0]["indicators"]["quote"][0]["close"][0:-14]
  # volume = payload["chart"]["result"][0]["indicators"]["quote"][0]["volume"][0:-14]

  # no need to strip the data when you run the pred at the same night 
  close = payload["chart"]["result"][0]["indicators"]["quote"][0]["close"]
  volume = payload["chart"]["result"][0]["indicators"]["quote"][0]["volume"]

  df1 = pd.DataFrame(close)
  df2 = pd.DataFrame(volume)
  data1 = df1.fillna(method='ffill')
  data2 = df2.fillna(0)
  # data1,data2 = newdf.filter(['Close']), newdf.filter(['Volume'])
  dataset1, dataset2 = data1.values, data2.values

  #scale the data
  scaler1 = MinMaxScaler(feature_range=(0,1))
  scaled_data1 = scaler1.fit_transform(dataset1)
  scaler2 = MinMaxScaler(feature_range=(0,1))
  scaled_data2 = scaler2.fit_transform(dataset2)

  #data Range for input (check your model)
  #dpRange = 56
  #dpRange = 48
  dpRange = 28

  dpSize = len(scaled_data1)
  test_data_len = 70
  sample_len = 5

  #create testing data set
  test_data1 = scaled_data1[0:test_data_len,:]
  test_data2 = scaled_data2[0:test_data_len,:]

  #create the datasets x_test and y_test
  trainFeature1 = []
  trainFeature2 = []
  trainFeature = []
  x_test = []
  y_train = []

  for i in range (dpSize - sample_len, dpSize):
  # for i in range (dpRange, dpSize):
    trainFeature1.append(test_data1[i-dpRange:i,0])
    trainFeature2.append(test_data2[i-dpRange:i,0])
    for j in range(0, len(trainFeature1)):
      tf = np.array((trainFeature1[j],trainFeature2[j]))

    tf = tf.T
    x_test.append(tf)
    y_train.append(dataset1[i,0])

    # if i<= dpRange:
    #   print(x_test)
    #   print(y_train)

  x_test, y_train = np.array(x_test), np.array(y_train)
  # x_test.shape

  #Calculate preds value using the model
  pred1  = model1.predict(x_test)
  pred1 = scaler1.inverse_transform(pred1)
  predPrice1 = np.round(pred1[:,0])
  # print(predPrice1)

  predFactor = predPrice1[4] - round(np.average(predPrice1))
  priceFactor = y_train[4] - round(np.average(y_train))
  #print(predFactor, priceFactor)
  #trend calculation

  trend = 0

  if(priceFactor > 0):
    if(predFactor > 0):
      trend = priceFactor + predFactor
    elif (predFactor == 0):
      trend =  priceFactor
    else:
      trend = (priceFactor + predFactor)/2
  elif(priceFactor == 0):
    if(predFactor > 0):
      trend = predFactor/2
    elif (predFactor == 0):
      trend =  0
    else:
      trend = predFactor/2
  else:
    if(predFactor > 0):
      trend = (priceFactor + predFactor)/2
    elif (predFactor == 0):
      trend =  priceFactor
    else:
      trend = priceFactor + predFactor
  
  trend = trend/predPrice1[4]    #make trend as percentage
  # print(trend)

  #get price & volume info
  url1 = ('https://query1.finance.yahoo.com/v7/finance/quote?lang=en-US&region=US&symbols='+ tickName[count] +
      '&fields=price%2CregularMarketVolume%2CaverageDailyVolume3Month%2CaverageDailyVolume10Day&corsDomain=finance.yahoo.com')
  # print(url)

  response1 = requests.get(url1)  # Make a GET request to the URL
  payload1 = response1.json()  # Parse `response.text` into JSON
  # print(payload1)
  mPrice = payload1["quoteResponse"]["result"][0]["regularMarketPrice"]
  mVolume = payload1["quoteResponse"]["result"][0]["regularMarketVolume"]
  avgVol90 = payload1["quoteResponse"]["result"][0]["averageDailyVolume3Month"]
  avgVol10 = payload1["quoteResponse"]["result"][0]["averageDailyVolume10Day"]

  rowIdx = 15  + count  
  wRange = "D" + str(rowIdx) + ":I" + str(rowIdx) 
  cellRange = ws.range(wRange)
  ll = 0
  for cell in cellRange:
    if(ll == 0):
      cell.value = str(mPrice)
    elif(ll == 1):
      cell.value = str(round(predPrice1[4]))
    elif(ll == 2):
      cell.value = str(trend)
    elif(ll == 3):
      cell.value = str(mVolume)
    elif(ll == 4):
      cell.value = str(avgVol90)
    else:
      cell.value = str(avgVol10)
    ll+=1

  ws.update_cells(cellRange, "RAW")
  #write result to gsheet
           # write pred result to worksheet staarting at rowIdx, predCol
  #update worksheet
  # ws.update_cell(rowIdx, 4, str(predPrice1))

